In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PowerTransformer

In [2]:
# Loading dataframe
df = pd.read_csv('London_Listings.csv')

# Original shape
print('Original Shape:', df.shape)

# First five rows
df.head()

Original Shape: (66679, 31)


,id,name,description,host_id,host_name,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,amenities,price,minimum_nights,maximum_nights,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,calculated_host_listings_count
0,2.763150e+07,1,NaN,35877912,Alex,15/06/2015,NaN,NaN,f,1.0,...,"[""Kitchen"", ""Refrigerator"", ""Microwave"", ""Free...",$200.00,1,1125,14/06/2024,0,NaN,NaN,NaN,1
1,1.100000e+18,1,Get comfortable and enjoy plenty of extra room...,564055882,Lili-Ann,26/02/2024,NaN,NaN,f,1.0,...,[],$675.00,1,365,15/06/2024,0,NaN,NaN,NaN,1
2,1.170000e+18,117,Keep it simple at this peaceful and centrally-...,528033284,Shujauddin,26/07/2023,86%,50%,f,2.0,...,"[""Washer"", ""Wifi"", ""Kitchen""]",$95.00,1,365,16/06/2024,0,NaN,NaN,NaN,2
3,8.730000e+17,- Bright Central London Apartment -,My apartment has everything you possibly need ...,99268099,Alex,11/10/2016,98%,93%,t,5.0,...,"[""Microwave"", ""Hot water kettle"", ""Body soap"",...",$166.00,7,365,16/06/2024,24,24/04/2023,02/05/2024,4.83,1
4,4.672403e+07,""" Bright & Cosy Loft apartment """,Lovely and cosy arctic space with a separate d...,359199241,Karol,29/07/2020,100%,95%,t,1.0,...,"[""Cleaning products"", ""Cooking basics"", ""First...",$105.00,2,60,14/06/2024,46,20/08/2021,02/06/2024,4.89,1


In [3]:
# Check types before categorising
print(df.dtypes)

id                                float64
name                               object
description                        object
host_id                             int64
host_name                          object
host_since                         object
host_response_rate                 object
host_acceptance_rate               object
host_is_superhost                  object
host_listings_count               float64
host_total_listings_count         float64
neighbourhood                      object
latitude                          float64
longitude                         float64
property_type                      object
room_type                          object
accommodates                        int64
bathrooms                         float64
bathrooms_text                     object
bedrooms                          float64
beds                              float64
amenities                          object
price                              object
minimum_nights                    

In [4]:
# Categorial and numerical columns
categorical_var = df.select_dtypes(include=['object'])
numerical_var = df.select_dtypes(include=['int64', 'float64'])

print("Categorical Variables:", categorical_var)
print("Numerical Variables:", numerical_var)

Categorical Variables:                                                  name  \
0                                                   1   
1                                                   1   
2                                                 117   
3                 - Bright Central London Apartment -   
4                   " Bright & Cosy  Loft apartment "   
...                                               ...   
66674  ZONE-1-PRIME LOCATION-LUXURY 1BR APT IN LONDON   
66675                                           Zone2   
66676                          Zoya’s Place in Harrow   
66677                                          #NAME?   
66678                                          #NAME?   

                                             description   host_name  \
0                                                    NaN        Alex   
1      Get comfortable and enjoy plenty of extra room...    Lili-Ann   
2      Keep it simple at this peaceful and centrally-...  Shujauddin   
3   

In [5]:
# Price Column
df['price'] = df['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False)
df['price'] = pd.to_numeric(df['price'], errors='coerce')

df['price'].describe()


count    61418.000000
mean       209.987202
std        961.450873
min          0.000000
25%         78.000000
50%        138.000000
75%        227.000000
max      80000.000000
Name: price, dtype: float64

In [6]:
# Categorical variables, remove empty strings or lists and replace with Nan.
for col in categorical_var:
    df[col] = df[col].replace(['', '[]'], np.nan)
    
missing_col_val = df.isnull().sum()
print('Missing column values:\n', missing_col_val[missing_col_val > 0])

Missing column values:
 description                   1956
host_name                        1
host_since                       1
host_response_rate           10175
host_acceptance_rate          7146
host_is_superhost              380
host_listings_count              1
host_total_listings_count        1
bathrooms                     5329
bathrooms_text                 100
bedrooms                       101
beds                          5400
amenities                      136
price                         5261
first_review                 14736
last_review                  14736
review_scores_rating         14736
dtype: int64


In [7]:
# Drop unnecessary columns, can be later changed.
drop_Columns = ['calendar_last_scraped', 'bathrooms_text', 'latitude', 'longitude']
df.drop(columns=drop_Columns, inplace=True, errors='ignore')

# Updated shape
print("Shape after dropping unnecessary columns:", df.shape)

Shape after dropping unnecessary columns: (66679, 27)


In [8]:
# Drop duplicates
df.drop_duplicates(inplace=True)

# Drop rows with NaN values
df.dropna(inplace=True)

# Updated shape
print('Shape after dropping duplicates and NaN rows:', df.shape)

Shape after dropping duplicates and NaN rows: (41888, 27)


In [9]:
# Remove outliers (ONLY RUN ONCE)

def remove_outliers(df, columns):
    for col in columns:
        # Necessary as numberical_var contains elements which have been deleted.
        if col not in df.columns:
            print(f"Column '{col}' does not exist. ")
            continue
        
        # Calculate IQR and bounds
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        l_bound = q1 - 1.5 * iqr
        u_bound = q3 + 1.5 * iqr
        
        # Remove outliers
        df = df[(df[col] >= l_bound) & (df[col] <= u_bound)]
    
    return df

# Call the function
df = remove_outliers(df, numerical_var)

# Display the updated shape
print('Shape after removing outliers:', df.shape)


Column 'latitude' does not exist. 
Column 'longitude' does not exist. 
Shape after removing outliers: (19275, 27)
